In [ ]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict,Literal,Annotated
from langchain_google_genai import ChatGoogleGenerativeAI
from google.generativeai as genai
from langchain_core.messages import SystemMessage, HumanMessage
import operator

In [ ]:
# Define evaluator_llm before using it
generator_llm = ChatGoogleGenerativeAI(model="gemini-pro")
evaluator_llm = ChatGoogleGenerativeAI(model="gemini-pro")
optimizer_llm = ChatGoogleGenerativeAI(model="gemini-pro")

In [ ]:
from pydantic import BaseModel, Field

class TweetEvaluation(BaseModel): 
    evaluation: Literal ["approved", "needs_improvement"] = Field (..., description="Final evaluation")
    feedback: str = Field(..., description-"feedback for the tweet.")

In [ ]:
structured_evaluator_llm = evaluator_llm.with_structured_output(TweetEvaluation)

In [ ]:
#state
class TweetState(TypedDict)

  topic : str
  tweet : str
  evaluation :Literal["approved", "need_improvement"]
  feedback:str
  iteration : int
  max_iteration : int
  
  tweet_history: Annotated[list[str],operator.add]
  feedback_history: Annotated[list[str],operator.add]

In [ ]:
def generate_tweet(state: TweetState):
    
    #prompt
    messages = [
        SystemMessage(content="You are a funnay and clever twitter/X influencer."),
        HumanMessage(content=f"""
    Write a short, orginal, and hilarious tweet on the topic:"{state['topic']}".
    
    Rules:
    -Do no use question_answer fromat.
    -Max 280 characters.
    -Use observational humor, irony, sarcasm, or cultural references.
    -think in meme logic, punchlines, or relatable task.
    -use simple, day to day english
    -This is version'{state['iteration'] + 1}.
    """)
    ]
    
    #send generator_llm
    response = generator_llm.invoke(messages).Content
    
    # Return response
    
    return {"tweet" : response, 'tweet_history':[response]}

In [ ]:
def evaluate_tweet(state:TweetState):
    
    
    #prompt
    messages = [
        SystemMessage(content= "You are a ruthless, no-laugh-given Twitter critic. You evaluate tweets based on humor, originally, virality, and tweet format."),
Eualuate the following tweet:
Tweet : "{state["tweet"]}"

Use the criteria below to evaluate the tweet:
1. Originality - Is this fresh , or have  you seen it a hundred times before ?
2. Humor - Did it genuinely make you smile, laugh, or chunkle?
3. Punchiness - Is it short, sharp,and scroll-stopping?
4. Virality Potential- would people retweet or share it ?
5. Format- Is it a well-formed tweet (not a setup-punchline joke, not a Q&A joke, and  under 20 characters)?

Auto-reject if :
-It's written in question-answer format (e.g., "Why did..." or "what happens when...")
-It exceeds 280 characters
-It reads like a traditional setup_punchline joke
-Dont end with generic, throwaway,or deflating lines that weaken the humar (e.g., "Masterpieces of the auntie-uncle
universe" or vague summaries)

##Responed only in structured  format:

- evaluation : "approved"  or "need_improvement
- feedback: One paragraph explaining the strengths  and weaknesses
""")
]
   
   
    response =  structured_evaluator_llm.invoke(messages) 
    
    return {'evaluation':response.evaluation, 'feedback' : response.feedback, 'feedback_history'[]}

In [ ]:
def optimize_tweet(state: TweetState):
    
    #prompt
    messages = [
        SystemMessage(conent="You punch up tweets for virality and humor based on given feedback."),
        HumanMessage(content= f"""
    Improve the tweet based on this feedback:
    "{state['feedback']}"
    
    Topic: "{state['topic']}
    Orginal Tweet:
    {state['tweet']}
    
    Re-write it as a short, viral-worthy tweet.Avoid Q&A style and stay under 280 characters.
    """)
    ]
    
    response= optimizer_llm.invoke(messages).content
    iteration = state['iteration'] + 1
    
    return{'tweet': response, 'iteration': iteration, 'tweet_history': [response]}

In [ ]:
def roote_evaluation(state: TweetState):
    
    if state['evaluation'] == 'approved' or state['iteration']>= state['max_iteration']:
        return 'apporved'
    else:
        return 'needs_improvement'

In [ ]:
graph = StateGraph(TweetState)

graph.add_node('generate' , generate_tweet)
graph.add_node('evaluate' , evaluate_tweet)
graph.add_node('optimize' , optimize_tweet)

graph.add_edge(START, 'generate')
graph.add_edge('generate' , 'evaluate')

#conditional edge
graph.add_conditional_edge('evaluate', route_evaluation, ('approved':END, 'needs_improvement':'optimize'))
graph.add_edge('optimize' , 'evaluate')

workflow = graph.compile()

workflow


In [ ]:
initial_state = {
    "topic" :"Pakistan Railways",
     "iteration":1,
     "max_iteration": 5
}

result = workflow.invoke(initial_state)

In [ ]:
result

In [ ]:
for tweet in result['tweet_history']:
    print(tweet)